In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [3]:
from companies.models import Company
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [4]:
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan

In [5]:
Company.objects.filter(num_employees__gt=F("num_chairs"))

<QuerySet [<Company: Company BBB>, <Company: Company CCC>]>

In [6]:
Company.objects.filter(num_employees__gt=F("num_chairs") * 2)

<QuerySet [<Company: Company CCC>]>

In [7]:
company = Company.objects.filter(
    num_employees__gt=F("num_chairs")
).annotate(
    chairs_needed=F("num_employees") - F("num_chairs")
).first()

In [8]:
print(company.chairs_needed)

20


In [10]:
from django.db.models import Value
from django.db.models.functions import Concat

companies = Company.objects.annotate(
    full_name = Concat("name", Value("("), "ticker", Value(")"))
)

for com in companies:
    print(com.full_name)

Company AAA(AAA)
Company BBB(BBB)
Company CCC(CCC)


In [3]:
comps = Company.objects.filter(
    num_employees__gt=F("num_chairs")
).annotate(
    chairs_needed=F("num_employees") - F("num_chairs")
).filter(
    chairs_needed__gt = 100
)

comps

<QuerySet []>

In [5]:
from books.models import *
Book.objects.filter(publisher__name="Penguin Books").count()

20

In [11]:
from django.db.models import Avg, Min, Max

res = Book.objects.aggregate(
    price_avg = Avg("price", default=0),
    price_min = Min("price", default=0),
    price_max = Max("price", default=0)
)
res

{'price_avg': Decimal('9.7018644067796610'),
 'price_min': Decimal('5.99'),
 'price_max': Decimal('14.99')}

In [10]:
Book.objects.filter(publisher__name = "Penguin Books").aggregate(Avg("price", default = 0))

{'price__avg': Decimal('9.6150000000000000')}

In [12]:
pubs = Publisher.objects.annotate(num_books = Count("book"))
for pub in pubs:
    print(f"PUB: {pub.name} => BOOK NUM: {pub.num_books}")

PUB: Oxford University Press => BOOK NUM: 39
PUB: Penguin Books => BOOK NUM: 20


In [13]:
pubs = Publisher.objects.annotate(avg_price = Avg("book__price"))
for pub in pubs:
    print(f"PUB: {pub.name} => AVG PRICE: {pub.avg_price}")

PUB: Oxford University Press => AVG PRICE: 9.7464102564102564
PUB: Penguin Books => AVG PRICE: 9.6150000000000000


In [14]:
from django.db.models import Q
above = Publisher.objects.annotate(above_4=Count("book", filter=Q(book__rating__gt=4)))
below = Publisher.objects.annotate(below_4=Count("book", filter=Q(book__rating__lte=4)))



PUB : Oxford University Press => BOOK NUM (RATING > 4): 33
PUB : Penguin Books => BOOK NUM (RATING > 4): 16


In [15]:
for item in above:
    print(f"PUB : {item.name} => BOOK NUM (RATING > 4): {item.above_4}")

PUB : Oxford University Press => BOOK NUM (RATING > 4): 33
PUB : Penguin Books => BOOK NUM (RATING > 4): 16


In [16]:
for item in below:
    print(f"PUB : {item.name} => BOOK NUM (RATING <= 4): {item.below_4}")

PUB : Oxford University Press => BOOK NUM (RATING <= 4): 6
PUB : Penguin Books => BOOK NUM (RATING <= 4): 4


In [18]:
pubs = Publisher.objects.annotate(num_books=Count("book")).order_by("num_books")
for pub in pubs:
    print(f"PUB: {pub.name} => BOOK NUM: {pub.num_books}")

PUB: Penguin Books => BOOK NUM: 20
PUB: Oxford University Press => BOOK NUM: 39


In [23]:
books = Book.objects.annotate(
    num_authors=Count("authors"),
    num_stores = Count("store")
).order_by("-num_stores", "-num_authors")[:10]

for book in books:
    print(f"NAME: {book.name}, NUM STORE: {book.num_stores}, NUM AUTHORS: {book.num_authors}")

NAME: Madame Bovary, NUM STORE: 1, NUM AUTHORS: 1
NAME: The Hobbit, NUM STORE: 1, NUM AUTHORS: 1
NAME: David Copperfield, NUM STORE: 1, NUM AUTHORS: 1
NAME: The Divine Comedy, NUM STORE: 1, NUM AUTHORS: 1
NAME: Pride and Prejudice, NUM STORE: 1, NUM AUTHORS: 1
NAME: Great Expectations, NUM STORE: 1, NUM AUTHORS: 1
NAME: Sense and Sensibility, NUM STORE: 1, NUM AUTHORS: 1
NAME: The Scarlet Letter, NUM STORE: 1, NUM AUTHORS: 1
NAME: Moby-Dick, NUM STORE: 1, NUM AUTHORS: 1
NAME: The Catch 22, NUM STORE: 1, NUM AUTHORS: 1


In [26]:
books = Book.objects.all().values("name", "pages", "price")[:3]
for book in books:
    print(book)

{'name': 'The Great Gatsby', 'pages': 180, 'price': Decimal('10.99')}
{'name': '1984', 'pages': 328, 'price': Decimal('8.99')}
{'name': 'To Kill a Mockingbird', 'pages': 281, 'price': Decimal('7.99')}
